In [ ]:
import ROOT

In [ ]:
import numpy as np
import scipy.optimize as optimize
import sys
!conda install --yes --prefix {sys.prefix} keras

In [ ]:
!wget -U 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.1.6) Gecko/20070802 SeaMonkey/1.1.4' https://arxiv.org/src/1411.4312v6/anc/lester_mt2_bisect.h
ROOT.gInterpreter.ProcessLine('#include "lester_mt2_bisect.h"')
myObj=ROOT.asymm_mt2_lester_bisect()
def calcMt2(V1,V2,t_sig):
    Met_X =(t_sig.met_et*np.cos(t_sig.met_phi))
    Met_Y = (t_sig.met_et*np.sin(t_sig.met_phi))
    return myObj.get_mT2(V1.M(),V1.Px(),V1.Py(),V2.M(),V2.Px(),V2.Py(),Met_X,Met_Y,600,600) 

In [ ]:
def filter(t_sig,sig_met_et,sig_mll,sig_mt2):#,sig_eta0,sig_eta1):
    print("loading data now")
    n = 0
    for event in t_sig:
        n += 1
        if(n%100000==0):
            print("Event ", n ,"out of ", t_sig.GetEntries())
        ##triggered by single electron or muon
        if(t_sig.trigE == True or t_sig.trigM == True): 
        
            ## exactly 2 SFOS leptons
            if(t_sig.lep_n == 2 and t_sig.lep_type[1] == t_sig.lep_type[0] and t_sig.lep_charge[0]+t_sig.lep_charge[1]==0): 
            
                #check energy of electrons and muons
                if(t_sig.lep_type[0] == 11 and t_sig.lep_pt[0] > 25) or (t_sig.lep_type[0] == 13 and t_sig.lep_pt[0] > 20): 
            
                        ########################MT2 CALC#########################                   
                    lep0 = ROOT.Math.PtEtaPhiEVector(t_sig.lep_pt[0],t_sig.lep_eta[0],t_sig.lep_phi[0],t_sig.lep_E[0])
                    lep1 = ROOT.Math.PtEtaPhiEVector(t_sig.lep_pt[1],t_sig.lep_eta[1],t_sig.lep_phi[1],t_sig.lep_E[1])
                    Sum = lep1 + lep0
                    mll = Sum.M()         
                    Mt2 = calcMt2(lep0,lep1,t_sig)  

                    sig_met_et.append((t_sig.met_et)/1000.)
                    sig_mll.append(mll)
                    sig_mt2.append(Mt2)
                    #sig_eta0.append(t_sig.lep_eta[0])
                    #sig_eta1.append(t_sig.lep_eta[1])
                        
                          
    print("Done!")

Upload MC Simulations

In [ ]:
## Reads data from MC simulation of slepton creation
sig = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_392999.SlepSlep_direct_700p0_300p0_2L8.2lep.root")
t_sig = sig.Get("mini")
print(t_sig.GetEntries(), " Entries for signal")

sig_met_et = []
sig_mll = []
sig_mt2 = []



In [ ]:
filter(t_sig,sig_met_et,sig_mll,sig_mt2)

In [ ]:
## DiBoson process ZZ -> qqll
bkg1 = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363356.ZqqZll.2lep.root")
t_bkg1 = bkg1.Get("mini")
print(t_bkg1.GetEntries()," Entries for background")

# DiBoson process WZ -> qqll
bkg2 = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363358.WqqZll.2lep.root")
t_bkg2 = bkg2.Get("mini")
print(t_bkg2.GetEntries()," Entries for background")

# DiBoson process WZ -> llvv
bkg3 = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363492.llvv.2lep.root")
t_bkg3 = bkg3.Get("mini")
print(t_bkg3.GetEntries()," Entries for background")

# DiBoson process w/ final state llll
bkg4 = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363490.llll.2lep.root")
t_bkg4 = bkg4.Get("mini")
print(t_bkg4.GetEntries()," Entries for background")

# DiBoson process w/ final state lllv
bkg5 = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363491.lllv.2lep.root")
t_bkg5 = bkg5.Get("mini")
print(t_bkg5.GetEntries()," Entries for background")



In [ ]:
bkg1_met_et = []
bkg1_mll = []
bkg1_mt2 = []

filter(t_bkg1,bkg1_met_et,bkg1_mll,bkg1_mt2)

In [ ]:
bkg2_met_et = []
bkg2_mll = []
bkg2_mt2 = []

filter(t_bkg2,bkg2_met_et,bkg2_mll,bkg2_mt2)

In [ ]:
bkg3_met_et = []
bkg3_mll = []
bkg3_mt2 = []

filter(t_bkg3,bkg3_met_et,bkg3_mll,bkg3_mt2)

In [ ]:
bkg4_met_et = []
bkg4_mll = []
bkg4_mt2 = []

filter(t_bkg4,bkg4_met_et,bkg4_mll,bkg4_mt2)

In [ ]:
bkg5_met_et = []
bkg5_mll = []
bkg5_mt2 = []

filter(t_bkg5,bkg5_met_et,bkg5_mll,bkg5_mt2)

Splitting data into training and testing

In [ ]:
##splitting data into training and testing
split = 1000

sigmid = int(len(sig_met_et)/2)
bkg1mid = int(len(bkg1_met_et)/split)
bkg2mid = int(len(bkg2_met_et)/split)
bkg3mid = int(len(bkg3_met_et)/split)
bkg4mid = int(len(bkg4_met_et)/split)
bkg5mid = int(len(bkg5_met_et)/split)
bkgmid = bkg1mid + bkg2mid + bkg3mid + bkg4mid + bkg5mid


#We want to make sure training data is about an even split of signal and bkg
print((len(bkg1_met_et) + len(bkg2_met_et) + len(bkg3_met_et) + len(bkg4_met_et) + len(bkg5_met_et))/1000)
print(len(sig_met_et))


Training data

In [ ]:
train_met = sig_met_et[:sigmid] + bkg1_met_et[:bkg1mid] + bkg2_met_et[:bkg2mid] + bkg3_met_et[:bkg3mid] + bkg4_met_et[:bkg4mid] + bkg5_met_et[:bkg5mid]
train_mll = sig_mll[:sigmid] + bkg1_mll[:bkg1mid] + bkg2_mll[:bkg2mid] + bkg3_mll[:bkg3mid] + bkg4_mll[:bkg4mid] + bkg5_mll[:bkg5mid]
train_mt2 = sig_mt2[:sigmid] + bkg1_mt2[:bkg1mid] + bkg2_mt2[:bkg2mid] + bkg3_mt2[:bkg3mid] + bkg4_mt2[:bkg4mid] + bkg5_mt2[:bkg5mid]
#train_eta0 = sig_eta0[:sigmid] + bkg1_eta0[:bkg1mid] + bkg2_eta0[:bkg2mid] + bkg3_eta0[:bkg3mid] + bkg4_eta0[:bkg4mid] + bkg5_eta0[:bkg5mid]
#train_eta1 = sig_eta1[:sigmid] + bkg1_eta1[:bkg1mid] + bkg2_eta1[:bkg2mid] + bkg3_eta1[:bkg3mid] + bkg4_eta1[:bkg4mid] + bkg5_eta1[:bkg5mid]


classessig=np.full((sigmid, 1), 1)
classesbkg=np.full((bkgmid, 1), 0)
classes = np.append(classessig,classesbkg)

training_data = np.array(list(zip(train_met,train_mll,train_mt2)))#,train_eta0,train_eta1)))


Testing Data

In [ ]:
test_met = sig_met_et[sigmid:] + bkg1_met_et[(split-1)*bkg1mid:] + bkg2_met_et[(split-1)*bkg2mid:] + bkg3_met_et[(split-1)*bkg3mid:] + bkg4_met_et[(split-1)*bkg4mid:] + bkg5_met_et[(split-1)*bkg5mid:]
test_mll = sig_mll[sigmid:] + bkg1_mll[(split-1)*bkg1mid:] + bkg2_mll[(split-1)*bkg2mid:] + bkg3_mll[(split-1)*bkg3mid:] + bkg4_mll[(split-1)*bkg4mid:] + bkg5_mll[(split-1)*bkg5mid:]
test_mt2 = sig_mt2[sigmid:] + bkg1_mt2[(split-1)*bkg1mid:] + bkg2_mt2[(split-1)*bkg2mid:] + bkg3_mt2[(split-1)*bkg3mid:] + bkg4_mt2[(split-1)*bkg4mid:] + bkg5_mt2[(split-1)*bkg5mid:]
#test_eta0 = sig_eta0[sigmid:] + bkg1_eta0[(split-1)*bkg1mid:] + bkg2_eta0[(split-1)*bkg2mid:] + bkg3_eta0[(split-1)*bkg3mid:] + bkg4_eta0[(split-1)*bkg4mid:] + bkg5_eta0[(split-1)*bkg5mid:]
#test_eta1 = sig_eta1[sigmid:] + bkg1_eta1[(split-1)*bkg1mid:] + bkg2_eta1[(split-1)*bkg2mid:] + bkg3_eta1[(split-1)*bkg3mid:] + bkg4_eta1[(split-1)*bkg4mid:] + bkg5_eta1[(split-1)*bkg5mid:]

test_data = np.array(list(zip(test_met,test_mll,test_mt2)))#,test_eta0,test_eta1)))


All MC simulations for Final Analysis

In [ ]:
Sig_data = np.array(list(zip(sig_met_et,sig_mll,sig_mt2)))#,sig_eta0,sig_eta1)))


bkg_met = bkg1_met_et + bkg2_met_et + bkg3_met_et + bkg4_met_et + bkg5_met_et
bkg_mll = bkg1_mll + bkg2_mll + bkg3_mll + bkg4_mll + bkg5_mll
bkg_mt2 = bkg1_mt2 + bkg2_mt2 + bkg3_mt2 + bkg4_mt2 + bkg5_mt2
#bkg_eta0 = bkg1_eta0 + bkg2_eta0 + bkg3_eta0 + bkg4_eta0 + bkg5_eta0
#bkg_eta0 = bkg1_eta1 + bkg2_eta1 + bkg3_eta1 + bkg4_eta1 + bkg5_eta1

Bkg_data = np.array(list(zip(bkg_met,bkg_mll,bkg_mt2)))#,bkg_eta0,bkg_eta1)))

Nueral Network Training

In [ ]:
# define a Keras model
def create_model(layers=(5,5), regularizer=None, activation='relu'):
    from keras.models import Sequential # feed-forward neural network (sequential layers)
    from keras.layers import Dense # fully interconnected layers
    model = Sequential()
    
    # input layer: 5 nodes by default (2 features to input to each node), relu activation. Weights will be initialized
    # with random values in a certain range
    model.add(Dense(layers[0], input_dim=3, activation=activation, kernel_regularizer=regularizer))
    
    # 2 hidden layers: 5 nodes by default
    model.add(Dense(layers[0], activation=activation, kernel_regularizer=regularizer))
    
    model.add(Dense(layers[0], activation=activation, kernel_regularizer=regularizer))
    
    # one output, mapped to [0,1] by sigmoid function
    model.add(Dense(1, activation='sigmoid'))
    
    # assemble the model (Translate to TensorFlow)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [ ]:
#check both are same shape
print(len(classes))
print(len(training_data))

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

model=KerasClassifier(build_fn=create_model, verbose=1)
results = model.fit(training_data, classes, epochs=40)

Graphing Results

In [ ]:
from keras.models import model_from_json
model.model.save("Training 4")
print("training saved")

In [ ]:
from matplotlib import pyplot
import matplotlib
matplotlib.rcParams['font.size'] = 16

trainprobs = model.predict_proba(training_data)[...,1]
testprobs = model.predict_proba(test_data)[...,1]

pyplot.figure(figsize = (20,8))
pyplot.subplot(1,2,1)
pyplot.title('Training Data')
pyplot.hist([np.extract(classes==0, trainprobs), np.extract(classes==1, trainprobs)], bins = 20,stacked=True, color=['tab:orange', 'tab:blue'])
pyplot.xlabel('Probability of event being signal')
pyplot.ylabel('# of points')
pyplot.yscale("log")
pyplot.xlim(0,1)


pyplot.subplot(1,2,2)
pyplot.title('Testing Data')
pyplot.hist([np.extract(classes==0, testprobs), np.extract(classes==1, testprobs)], bins = 20,stacked=True, color=['tab:orange', 'tab:blue'])
pyplot.xlabel('Probability of event being signal')
pyplot.ylabel('# of points')
pyplot.yscale("log")
pyplot.xlim(0,1)

In [ ]:
#Period A
ATLASfileA = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/Data/data_A.2lep.root")
ATLASA = ATLASfileA.Get("mini")
print(ATLASA.GetEntries(), " Entries for signal")

#Period B
ATLASfileB = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/Data/data_B.2lep.root")
ATLASB = ATLASfileB.Get("mini")
print(ATLASB.GetEntries(), " Entries for signal")

#Period C
ATLASfileC = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/Data/data_C.2lep.root")
ATLASC = ATLASfileC.Get("mini")
print(ATLASC.GetEntries(), " Entries for signal")

#Period D
ATLASfileD = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/Data/data_D.2lep.root")
ATLASD = ATLASfileD.Get("mini")
print(ATLASD.GetEntries(), " Entries for signal")

In [ ]:
ATLAS_met = []
ATLAS_mll = []
ATLAS_mt2 = []

filter(ATLASA,ATLAS_met,ATLAS_mll,ATLAS_mt2)

In [ ]:
filter(ATLASB,ATLAS_met,ATLAS_mll,ATLAS_mt2)# Scales Atlas data

In [ ]:
filter(ATLASC,ATLAS_met,ATLAS_mll,ATLAS_mt2)# Scales Atlas data

In [ ]:
filter(ATLASD,ATLAS_met,ATLAS_mll,ATLAS_mt2)# Scales Atlas data

In [ ]:
ATLAS_data = np.array(list(zip(ATLAS_met,ATLAS_mll,ATLAS_mt2)))

In [ ]:
ATLASprob = model.predict_proba(ATLAS_data)[...,1]


In [ ]:
Sigprob = model.predict_proba(Sig_data)[...,1]

In [ ]:
Bkgprob = model.predict_proba(Bkg_data)[...,1]

In [ ]:
h_bkg = ROOT.TH1F("h_bgs","Met_et",20,0,2000)

h_sig = ROOT.TH1F("h_sig","Met_et",20,0,2000)

h_ATLAS = ROOT.TH1F("h_ATLAS","Met_et",20,0,2000)


In [ ]:
for i in range(len(Sigprob)):
    if(Sigprob[i] > 0.85):
        h_sig.Fill(sig_met_et[i])

for i in range(len(Bkgprob)):
    if(Bkgprob[i] > 0.85):
        h_bkg.Fill(bkg_met[i])


for i in range(len(ATLAS_met)):
    if(ATLASprob[i] > 0.85):
        h_ATLAS.Fill(ATLAS_met[i])
        
        

In [ ]:
c = ROOT.TCanvas("testCanvas","a first way to plot a variable",800,600)

h_bkg.SetFillStyle(3001)
h_bkg.SetFillColor(4)
h_bkg.SetLineColor(4)

h_sig.SetFillStyle(3003)
h_sig.SetFillColor(2)
h_sig.SetLineColor(2)

h_ATLAS.SetFillStyle(3001)
h_ATLAS.SetFillColor(1)
h_ATLAS.SetLineColor(1)

legend=ROOT.TLegend(0.5,0.7,0.9,0.9)
legend.AddEntry(h_bkg,"Background (sl) ","l")
legend.AddEntry(h_sig,"Signal (Z #rightarrow ll)","l")
legend.AddEntry(h_ATLAS,"Data","l")

h_sig.SetStats(0)
h_bkg.SetStats(0)
h_ATLAS.SetStats(0)


h_ATLAS.SetMaximum(1e4)
h_ATLAS.SetMinimum(1e-1)
h_ATLAS.Draw("E1")
h_sig.Draw("histsame")
h_bkg.Draw("histsame")


legend.Draw()
c.SetLogy()
c.Draw()